<a href="https://colab.research.google.com/github/Hirva-Bhagat/GooglePlaceReviewScraper/blob/master/LSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.decomposition import TruncatedSVD
# If nltk stop word is not downloaded
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
import nltk,csv,numpy
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize, pos_tag
sentences=[]
reader = csv.reader(open('/content/drive/My Drive/TWWReviews.csv', 'rU'), delimiter= ",",quotechar='|')
for line in reader:
    for field in line:
      sentences.append(field)
print(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 'U' mode is deprecated
  """


['UserName', 'Reviews', 'Rating', 'Date', 'Justine Soucy', '"THIS IS MY HAPPY PLACE. I\'ve been wanting to visit Hogwarts ever since it was announced that this theme park was in construction. ', 'The decor', ' ambiance', ' music', ' and details are all so great and well executed. ', 'The best thing ever for a fan …"', ' 5 stars ', '3 months ago', 'Nathan Estes', '"Extra ordinary place to hangout with kids with numerous rides such as roller coasters 🎢', ' pacific wheel and many more to enjoy late night. It also has many delicious food 🥘 stalls."', ' 5 stars ', '3 months ago', 'Keyla Chamay Pioquinto', '"The place is awesome ! Big fan of the whole franchise ', 'wish I could be there again some time soon ! 😝❤️"', ' 5 stars ', 'a month ago', 'Louis Nava', '"I\'m not enthusiastic about this place at all. It\'s super basic; I would give it a 3 star on the food but the price to quality ratio is dragging this one down to a 2 star rating.', 'Let\'s talk about the fail backward on the food. By t

In [5]:
df = pd.DataFrame()
df["documents"] = sentences
df.head()
df['clean_documents'] = df['documents'].str.replace("[^a-zA-Z#]", " ")
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: x.lower())

df.head()

,documents,clean_documents
0,UserName,username
1,Reviews,reviews
2,Rating,rating
3,Date,date
4,Justine Soucy,justine soucy


In [ ]:
# tokenization
tokenized_doc = df['clean_documents'].fillna('').apply(lambda x: x.split())

# remove stop-words

tokenized_doc = tokenized_doc.apply(lambda x: [itemstop_words = stopwords.words('english')
 for item in x if item not in stop_words])

# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

df['clean_documents'] = detokenized_doc

In [11]:
# TF-IDF vector
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
X = vectorizer.fit_transform(df['clean_documents'])
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(X)


In [13]:
#Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = ["topic_1", "topic_2"])
topic_encoded_df["documents"] = df['clean_documents']
display(topic_encoded_df[["documents", "topic_1", "topic_2"]])

,documents,topic_1,topic_2
0,username,-0.0000000000000000,0.0000000000000000
1,reviews,-0.0000000000000000,-0.0000000000000000
2,rating,0.0000000003231811,0.0000000547193279
3,date,0.0000000000000000,0.0000000000000000
4,justine soucy,-0.0000000000000000,-0.0000000000000000
...,...,...,...
2745,months ago,0.0000000111347292,0.5766979505981678
2746,ella von pesso,0.0000000000000000,0.0000000000000000
2747,awesome experience very cool attractions and l...,0.0000003243162112,0.0000942800311642
2748,stars,0.9999996533796837,-0.0000000518495076


In [14]:
dictionary = vectorizer.get_feature_names()

In [15]:
encoding_matrix = pd.DataFrame(svd_model.components_, index = ["topic_1","topic_2"], columns = (dictionary)).T
encoding_matrix

,topic_1,topic_2
aaron,0.0000000000034377,0.0000000000667240
abbas,-0.0000000000000000,-0.0000000000000000
abbie,-0.0000000000000000,-0.0000000000000000
abe,0.0000000000000000,-0.0000000000000000
abina,-0.0000000000000000,-0.0000000000000000
...,...,...
zaid,0.0000000000000000,0.0000000000000000
zawahri,0.0000000000000000,0.0000000000000000
zayas,0.0000000000000000,0.0000000000000000
zero,0.0005287573610513,0.0000001181907852
